<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/07Jul2022_MLB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -q https://github.com/profteachkids/StemUnleashed/raw/main/mlb_batting.zip -O mlb.zip

In [2]:
import shutil
from pathlib import Path
import pandas as pd

In [3]:
shutil.unpack_archive('mlb.zip', extract_dir='mlb')

In [4]:
files=list(Path('mlb').glob('**/*.csv'))
keys=[afile.as_posix().split('/')[1:] for afile in files]
keys=[tuple((key[0], key[1], key[-1].split('.')[0])) for key in keys]

In [5]:
df_list=[]
days=['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
for afile in files:
    df=pd.read_csv(afile).iloc[:,1:]
    df['home']=df['OPP'].str.startswith('vs').astype(int)
    df['win']=(df['Result'].str[0]=='W').astype(int)
    df=df.loc[df['Date'].str[:3].isin(days),['Date','AB','HR','home','win']]
    df[['day','Date']]=df.loc[:,'Date'].str.split(' ',expand=True)
    df_list.append(df)

In [6]:
df=pd.concat(df_list,keys=keys)
df.index=df.index.droplevel(3)
df.index.names=['team','player','year']
df=df.reset_index().set_index(['team','player'])
df['date']=pd.to_datetime(df['Date'] + '/' + df['year'])
df=df.drop(['year','Date'],axis=1)

In [7]:
df

AB HR  home  win  day       date
team                player                                          
los-angeles-dodgers Freddie Freeman  4  0     1    0  Wed 2019-10-09
                    Freddie Freeman  5  0     0    0  Mon 2019-10-07
                    Freddie Freeman  4  0     0    1  Sun 2019-10-06
                    Freddie Freeman  3  0     1    1  Fri 2019-10-04
                    Freddie Freeman  4  1     1    0  Thu 2019-10-03
...                                 .. ..   ...  ...  ...        ...
pittsburgh-pirates  Oneil Cruz       5  0     1    0  Wed 2022-06-22
                    Oneil Cruz       4  0     1    1  Tue 2022-06-21
                    Oneil Cruz       5  0     1    1  Mon 2022-06-20
                    Oneil Cruz       4  1     1    0  Sun 2021-10-03
                    Oneil Cruz       5  0     1    1  Sat 2021-10-02

[252919 rows x 6 columns]

In [8]:
df[['AB','HR']]=df[['AB','HR']].astype(int)

In [9]:
df

AB  HR  home  win  day       date
team                player                                            
los-angeles-dodgers Freddie Freeman   4   0     1    0  Wed 2019-10-09
                    Freddie Freeman   5   0     0    0  Mon 2019-10-07
                    Freddie Freeman   4   0     0    1  Sun 2019-10-06
                    Freddie Freeman   3   0     1    1  Fri 2019-10-04
                    Freddie Freeman   4   1     1    0  Thu 2019-10-03
...                                  ..  ..   ...  ...  ...        ...
pittsburgh-pirates  Oneil Cruz        5   0     1    0  Wed 2022-06-22
                    Oneil Cruz        4   0     1    1  Tue 2022-06-21
                    Oneil Cruz        5   0     1    1  Mon 2022-06-20
                    Oneil Cruz        4   1     1    0  Sun 2021-10-03
                    Oneil Cruz        5   0     1    1  Sat 2021-10-02

[252919 rows x 6 columns]

In [28]:
team_HR_ratio=df.pivot_table(index=['team'],values=['AB','HR'],columns='home',aggfunc='sum')
team_HR_ratio=team_HR_ratio.rename(columns={0:"Away",1:"Home"},level=1)
team_HomeAway=team_HR_ratio['HR']/team_HR_ratio['AB']
team_HomeAway['ratio']=team_HomeAway['Home']/team_HomeAway['Away']
team_HomeAway.sort_values('ratio',ascending=False)

home,Away,Home,ratio
team,,,
colorado-rockies,0.033055,0.042727,1.292609
chicago-white-sox,0.032907,0.039290,1.193999
baltimore-orioles,0.035093,0.041690,1.187971
san-diego-padres,0.035341,0.041202,1.165843
los-angeles-dodgers,0.038326,0.044121,1.151208
seattle-mariners,0.039068,0.044925,1.149908
boston-red-sox,0.035879,0.040978,1.142131
houston-astros,0.032262,0.036200,1.122068
cincinnati-reds,0.035382,0.039470,1.115548


In [52]:
player_HR_ratio=df.pivot_table(index=['player'],values=['AB','HR'],columns='home',aggfunc='sum')
player_HR_ratio=player_HR_ratio.rename(columns={0:"Away",1:"Home"},level=1)
player_HomeAway=player_HR_ratio['HR']/player_HR_ratio['AB']
player_HomeAway=player_HomeAway.query('Away>0 and Home>0')  #avoid division by zero

In [53]:

player_HomeAway['ratio']=player_HomeAway['Home']/player_HomeAway['Away']
player_HomeAway=player_HomeAway.join(player_HR_ratio['AB'],rsuffix='AB')
player_HomeAway.sort_values('ratio',ascending=False)

home,Away,Home,ratio,AwayAB,HomeAB
player,,,,,
Matt Thaiss,0.011905,0.086957,7.304348,84,92
Matt Reynolds,0.005263,0.034091,6.477273,190,176
Jake Burger,0.010989,0.070175,6.385965,91,114
Gavin Sheets,0.017143,0.089172,5.201699,175,157
Riley Adams,0.010526,0.048780,4.634146,95,82
...,...,...,...,...,...
Eli White,0.041176,0.011050,0.268350,170,181
Austin Wynns,0.048485,0.010989,0.226648,165,182
Keibert Ruiz,0.031646,0.006494,0.205195,158,154


home,Away,Home,ratio,AwayAB,HomeAB
player,,,,,
AJ Pollock,0.034342,0.041855,1.218767,1718,1768
Aaron Hicks,0.030501,0.040639,1.332366,1377,1378
Aaron Judge,0.069804,0.089030,1.275433,1275,1258
Abraham Toro,0.031250,0.034268,1.096573,384,321
Adalberto Mondesi,0.029827,0.030960,1.037966,637,646
...,...,...,...,...,...
Yoshi Tsutsugo,0.039370,0.030534,0.775573,254,262
Yu Chang,0.031963,0.025974,0.812616,438,308
Yuli Gurriel,0.026993,0.036987,1.370248,1593,1487


In [48]:
player_HR_ratio

AB         HR     
home              Away  Home Away Home
player                                
AJ Pollock        1718  1768   59   74
Aaron Hicks       1377  1378   42   56
Aaron Judge       1275  1258   89  112
Aaron Whitefield     5     7    0    0
Abraham Toro       384   321   12   11
...                ...   ...  ...  ...
Yu Chang           438   308   14    8
Yuli Gurriel      1593  1487   43   55
Zach McKinstry      85    87    2    6
Zach Reks           22    22    0    0
Zack Collins       201   163    8    3

[510 rows x 4 columns]